In [27]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
## load mnist dataset

root = './data'
if not os.path.exists(root):
    os.mkdir(root)
    
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)

batch_size = 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

print ('==>>> total trainning batch number: {}'.format(len(train_loader)))
print ('==>>> total testing batch number: {}'.format(len(test_loader)))

==>>> total trainning batch number: 600
==>>> total testing batch number: 100


In [28]:
train_loader

In [29]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def name(self):
        return "LeNet"

## training
model = LeNet()

In [30]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

criterion = nn.CrossEntropyLoss()

In [31]:
for epoch in range(10):
    # trainning
    ave_loss = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        ave_loss = ave_loss * 0.9 + loss.data[0] * 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print ('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(epoch, batch_idx+1, ave_loss))
    # testing
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        x, target = Variable(x, volatile=True), Variable(target, volatile=True)
        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        # smooth average
        ave_loss = ave_loss * 0.9 + loss.data[0] * 0.1
        
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_loader):
            print ('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(epoch, batch_idx+1, ave_loss, correct_cnt * 1.0 / total_cnt))

torch.save(model.state_dict(), model.name())


/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # This is added back by InteractiveShellApp.init_path()


==>>> epoch: 0, batch index: 100, train loss: 0.572324
==>>> epoch: 0, batch index: 200, train loss: 0.261136
==>>> epoch: 0, batch index: 300, train loss: 0.175545
==>>> epoch: 0, batch index: 400, train loss: 0.153912
==>>> epoch: 0, batch index: 500, train loss: 0.098054
==>>> epoch: 0, batch index: 600, train loss: 0.105519


/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/apple/anaconda3/envs/ztdl/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


==>>> epoch: 0, batch index: 100, test loss: 0.061113, acc: 0.000
==>>> epoch: 1, batch index: 100, train loss: 0.091491
==>>> epoch: 1, batch index: 200, train loss: 0.079549
==>>> epoch: 1, batch index: 300, train loss: 0.067319
==>>> epoch: 1, batch index: 400, train loss: 0.064144
==>>> epoch: 1, batch index: 500, train loss: 0.074791
==>>> epoch: 1, batch index: 600, train loss: 0.054066
==>>> epoch: 1, batch index: 100, test loss: 0.052294, acc: 0.000
==>>> epoch: 2, batch index: 100, train loss: 0.057003
==>>> epoch: 2, batch index: 200, train loss: 0.045149
==>>> epoch: 2, batch index: 300, train loss: 0.043695
==>>> epoch: 2, batch index: 400, train loss: 0.050808
==>>> epoch: 2, batch index: 500, train loss: 0.039839
==>>> epoch: 2, batch index: 600, train loss: 0.042638
==>>> epoch: 2, batch index: 100, test loss: 0.033482, acc: 0.000
==>>> epoch: 3, batch index: 100, train loss: 0.036857
==>>> epoch: 3, batch index: 200, train loss: 0.039556
==>>> epoch: 3, batch index: 300